<a href="https://colab.research.google.com/github/Navodhya-2011/AdminDashboard/blob/main/Chat_with_PDF_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain transformers llama-index PyPDF2 sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [25]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Upload your PDF
from google.colab import files
uploaded = files.upload()

# Extract text from the uploaded PDF
pdf_path = list(uploaded.keys())[0]
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text[:500])  # Preview first 500 characters


Saving HR Handbook 2022 V8.pdf to HR Handbook 2022 V8 (2).pdf
DFCC BANK EMPLOYEE HANDBOOKLet’s Growas One 
2
 3CONTENT S
CEO’S Mes sage ............................................................................................................................... ........................................................................... 6
Our S tory 8
1.1 Who W e Are ............................................................................................................................... ...............................................................


In [31]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks = text_splitter.split_text(pdf_text)


In [32]:
from sentence_transformers import SentenceTransformer

# Load a SentenceTransformer model for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for each chunk
embeddings = [embedding_model.encode(chunk) for chunk in chunks]


In [33]:
import numpy as np

# Simulated embedding store
chunk_store = {i: chunks[i] for i in range(len(chunks))}
embedding_store = {i: embeddings[i] for i in range(len(chunks))}


In [34]:
from scipy.spatial.distance import cosine

def find_relevant_chunk(query, embedding_store):
    query_embedding = embedding_model.encode(query)
    similarities = {
        i: 1 - cosine(query_embedding, emb) for i, emb in embedding_store.items()
    }
    best_match = max(similarities, key=similarities.get)
    return best_match, chunk_store[best_match]


In [35]:
def generate_response(query, context, model, tokenizer):
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [41]:
!pip install transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the model and tokenizer
model_name = "facebook/bart-large-cnn" # Example model

# Check if CUDA is available and move model to the appropriate device
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Warning: CUDA not found. Using CPU instead.")

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def generate_response(query, context, model, tokenizer):
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device) # Send inputs to the correct device
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def chat_with_pdf(query):
    best_match_idx, context = find_relevant_chunk(query, embedding_store)
    response = generate_response(query, context, model, tokenizer)
    return response

# Example query
query = "Your Duties and Responsibilities?"
response = chat_with_pdf(query)
print(response)

14EMPL OYMENT RULES AND REGULA TIONS2152.1 Your Duties and Responsibilities. Your duties and responsibilities will be assigned to you in keeping with your contract/terms of employment. The Bank reserves the right to vary these duties/responsibilities from time to time.
